# A Tour of Oumi

Welcome to Open Universal Machine Intelligence (Oumi)! We created this platform to democratize the development of large models for the open source world. We strongly believe that together, as a community, we can push the boundaries of AI.

This tutorial will give you a brief overview of Oumi's core functionality. We'll cover:

1. Training a model
1. Performing model inference
1. Evaluating a model against common benchmarks
1. Launching jobs
1. Customizing datasets and clouds

# Prerequisites
## Oumi Installation
First, let's install Oumi. You can find detailed instructions [here](https://github.com/oumi-ai/oumi/blob/main/README.md), but it should be as simple as:

```bash
pip install -e ".[dev]"
```

## Creating our working directory
For our experiments, we'll use the following folder to save the model, training artifacts, and our working configs.

In [1]:
from pathlib import Path

tutorial_dir = "tour_tutorial"

Path(tutorial_dir).mkdir(parents=True, exist_ok=True)

# Training a model

Oumi supports training both custom and out-of-the-box models. Want to try out a model on HuggingFace? We can do that. Want to train your own custom Pytorch model? We've got you covered there too.

## A quick demo

Let's try training a pre-existing model on HuggingFace. We'll use GPT2 as it's small and trains quickly.

Oumi uses [training configuration files](https://oumi.ai/docs/latest/api/oumi.core.configs.html#oumi.core.configs.TrainingConfig) to specify training parameters. We've already created a training config for GPT2--let's give it a try!

In [ ]:
%%writefile $tutorial_dir/train.yaml

model:
  model_name: "gpt2" # 124M params.
  model_max_length: 128
  torch_dtype_str: "bfloat16"
  load_pretrained_weights: False
  trust_remote_code: True
  model_kwargs:
    disable_dropout: True

data:
  train:
    datasets:
      - dataset_name: "HuggingFaceFW/fineweb-edu"
        subset: "sample-10BT"
        split: "train"
    stream: True
    pack: True
    target_col: "text"

training:
  trainer_type: "TRL_SFT"
  per_device_train_batch_size: 2
  max_steps: 10 # Quick "mini" training, for demo purposes only.

  enable_gradient_checkpointing: False
  gradient_checkpointing_kwargs:
    use_reentrant: False

  learning_rate: 6.0e-04
  lr_scheduler_type: "cosine_with_min_lr"
  lr_scheduler_kwargs:
    min_lr_rate: 0.1
  warmup_steps: 715
  adam_beta1: 0.9
  adam_beta2: 0.95
  weight_decay: 0.1

  run_name: "gpt2_pt"


In [ ]:
from oumi.core.configs import TrainingConfig
from oumi.train import train

config = TrainingConfig.from_yaml(str(Path(tutorial_dir) / "train.yaml"))
config.training.output_dir = str(Path(tutorial_dir) / "output")

train(config)

Congratulations, you've trained your first model using Oumi!

You can also train your own custom Pytorch model. We cover that in depth in our [Finetuning Tutorial](https://github.com/oumi-ai/oumi/blob/main/notebooks/Oumi%20-%20Finetuning%20Tutorial.ipynb).

# Model Inference

Now that you've trained a model, let's run inference.

In [ ]:
%%writefile $tutorial_dir/train_inference_config.yaml

model:
  model_name: "tour_tutorial/output"
  trust_remote_code: true
  torch_dtype_str: "half"
  device_map: "auto"

generation:
  max_new_tokens: 128
  batch_size: 1

In [ ]:
from oumi.core.configs import InferenceConfig
from oumi.infer import infer

config = InferenceConfig.from_yaml(
    str(Path(tutorial_dir) / "train_inference_config.yaml")
)

input_text = (
    "Remember that we didn't train for long, so the results might not be great."
)

results = infer(config=config, inputs=[input_text])

print(results[0])

We can also run inference using the pretrained model by slightly tweaking our config:

In [ ]:
pretrained_config = InferenceConfig.from_yaml(
    str(Path(tutorial_dir) / "train_inference_config.yaml")
)
pretrained_config.model.model_name = "gpt2"

input_text = "Input for the pretrained model: What is your name? "

results = infer(config=pretrained_config, inputs=[input_text])

print(results[0])

# Evaluating a model against common benchmarks

You can use Oumi to evaluate pretrained and tuned models against standard benchmarks. For example, let's evaluate the pretrained version of our GPT2 model against `Hellaswag`:

In [ ]:
%%writefile $tutorial_dir/eval.yaml

model:
  model_name: "gpt2"
  trust_remote_code: True

lm_harness_params:
  tasks:
    - "hellaswag"

generation:
  batch_size: 0  # This will let LM HARNESS decide.


In [ ]:
from oumi.core.configs import EvaluationConfig
from oumi.evaluate import evaluate

eval_config = EvaluationConfig.from_yaml(str(Path(tutorial_dir) / "eval.yaml"))

# Uncomment the following line to run evals against the V1 HuggingFace Leaderboard.
# This may take a while.
# eval_config.data.datasets[0].dataset_name = "huggingface_leaderboard_v1"

evaluate(eval_config)  # EleutherAI Harness reports acc 28.92%, acc_norm 31.14%.

# Launching Jobs

Often times you'll need to run various tasks (training, evaluation, etc.) on remote hardware that's better suited for the task. Oumi can handle this for you by launching jobs on various compute clusters. For more information about running jobs, see our [Running Jobs Remotely tutorial](https://github.com/oumi-ai/oumi/blob/main/notebooks/Oumi%20-%20Running%20Jobs%20Remotely.ipynb). For running jobs on custom clusters, see our [Launching Jobs on Custom Clusters tutorial](https://github.com/oumi-ai/oumi/blob/main/notebooks/Oumi%20-%20Launching%20Jobs%20on%20Custom%20Clusters.ipynb).


Today, Oumi supports running jobs on several cloud provider platforms.

For the latest list, we can run the `which_cloud` method:

In [ ]:
import oumi.launcher as launcher

print("Supported Clouds in Oumi:")
for cloud in launcher.which_clouds():
    print(cloud)

Let's run a simple "Hello World" job locally to demonstrate how to use the Oumi job launcher. This job will echo `Hello World`, then run the same GPT2 training job executed above.

In [ ]:
%%writefile $tutorial_dir/job.yaml

name: hello-world
resources:
  cloud: local

# Upload working directory to remote.
working_dir: .

envs:
  TEST_ENV_VARIABLE: '"Hello, World!"'
  OUMI_LOGGING_DIR: "tour_tutorial/logs"


run: |
  set -e  # Exit if any command failed.

  echo "$TEST_ENV_VARIABLE"
  oumi train -c tour_tutorial/train.yaml


In [ ]:
import time

job_config = launcher.JobConfig.from_yaml(str(Path(tutorial_dir) / "job.yaml"))
cluster, job = launcher.up(job_config, cluster_name=None)

while job and not job.done:
    print("Job is running...")
    time.sleep(5)
    job = cluster.get_job(job.id)
print("Job is done!")

The job created logs under `/tour_tutorial/logs`. Let's take a look:

In [ ]:
logs_dir = Path(tutorial_dir) / "logs"
for log_file in logs_dir.iterdir():
    print(f"Log file: {log_file}")
    with open(log_file) as f:
        print(f.read())

# Customizing datasets and clusters

Oumi offers rich customization that allows users to build custom solutions on top of our existing building blocks. Several of Oumi's primary resources (Datasets, Clouds, etc.) leverage the Oumi Registry when invoked.

This registry allows users to build custom classes that function as drop-in replacements for core functionality.

For more details on registering custom datasets, see the [demo here](https://oumi.ai/docs/latest/advanced/custom_datasets.html).

For a tutorial on writing a custom cloud/cluster for running jobs, see the [tutorial here](https://github.com/oumi-ai/oumi/blob/main/notebooks/Oumi%20-%20Launching%20Jobs%20on%20Custom%20Clusters.ipynb).

You can find further information about the required registry decorators [here](https://oumi.ai/docs/latest/api/oumi.core.registry.html#oumi.core.registry.register_cloud_builder).

# What's next?

Now that you've completed the basic tour, you're ready to tackle our other [notebook guides](https://github.com/oumi-ai/oumi/tree/main/notebooks). 

Make sure you also take a look at our [Quickstart](https://oumi.ai/docs/latest/get_started/quickstart.html) for an overview of our CLI.
